In [1]:
from bs4 import BeautifulSoup #For web scraping
import requests 
import unicodedata
import re
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
#FIrst step is getting the data?
#Or do you want to see what all i've done first? done first means?
#Till now what i have done.
#SUrE
#I'm going to go one step ata  time

In [3]:
#Times of India 
url = "http://timesofindia.indiatimes.com/"
newsheadings = []
r  = requests.get(url)
data = r.text
soup = BeautifulSoup(data)

lists = soup.findAll('ul', attrs={'data-vr-zone': {"top_stories","latest","ots"}})

for i in lists:
    links = i.findAll('a')
    for link in links:
        title = link.get('title')
        if title:
            newsheadings.append(title.encode('ascii','ignore'))

/usr/local/lib/python2.7/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /usr/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [4]:
#Indian Express
url = "http://indianexpress.com/"
r  = requests.get(url)
data = r.text
soup = BeautifulSoup(data)

lists = soup.findAll('div', attrs={'class': 'top-news'})

for item in lists:
    links = item.findAll('a')
    for data1 in links:
        string = str(data1)
        m = re.search('>.*<', string)
        newsheadings.append( m.group(0)[1:-1])

In [5]:
#NDTV
url = "http://www.ndtv.com/"
r  = requests.get(url)
data = r.text
soup = BeautifulSoup(data)
lists = soup.findAll('a', attrs={'class': "item-title"})
for data1 in lists:
    string = unicode(data1).encode('ascii','ignore')
    m = re.search('>.*<', string)
    if len(m.group(0)[1:-1].split()) > 5:
        newsheadings.append(m.group(0)[1:-1])

In [6]:
#len(newsheadings)
news2 = list(set(newsheadings))

In [7]:
news2
#This is all the news headings. Totally from all channels.

['<strong>This Is Why You Crave Sugar Immediately After Your Meal</strong>',
 'Kareena Had A Visitor At Work - Taimur. He Just Dropped By, You Know',
 'Awkward. OPS, 3-Time Tamil Nadu Chief Minister, Now Deputy To EPS',
 "Live: SC verdict on triple talaq 'historic', says PM",
 'What happens to us, ask recent triple talaq victims',
 " Hey Govinda, Here's What David Dhawan Said About Working With You Again",
 'SC declares triple talaq unconstitutional',
 "McDonald's Terminates Franchise Agreement For 169 Outlets In India",
 'Justin Trudeau Goes Desi, Wears Kurta At India Day Parade. See Pics',
 'Triple talaq illegal now, no need for legislation: Shahnawaz\xc2\xa0Hussain',
 'Dengue Claims Its First Victim In Delhi - 12-Year-Old Boy From Bihar',
 'Pro Kabaddi 2017: After Kabaddi success, TransStadia eyes to host more sports in\xc2\xa0Ahmedabad',
 'How India Celebrated Organ Donation Week',
 'Delhi court denies bail to Kashmiri separatist leader Shabir\xc2\xa0Shah',
 'Sidharth-Alia break-up

In [8]:
len(news2)
#Screw from here. I'll have to explain later. MAybe incollege? This is the preprocess part
#Just look at thescraping part

143

In [10]:
lower= []
tokenised = [] 
final_preprocess = []
stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()

for i in news2:
    i_new = "".join(c for c in i if c not in punctuation)
    lower.append(i_new.strip(',.').lower())
    
for i in lower:
    tokenised.append(i.split())
    
for i in tokenised:
    for w in i:
        if w in stop_words:
            i.remove(w)
            
for i in tokenised:
    list1 = []
    for w in i:
        list1.append(str(w))#str(wordnet_lemmatizer.lemmatize(w)))
    final_preprocess.append(' '.join(list1))
    


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
vectors = tfidf.fit_transform(final_preprocess)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
mat = cosine_similarity(vectors, vectors)

In [13]:
removed = []
buckets = []
for i in range (0,mat.shape[0]):
    l1 = []
    for j in range (i,mat.shape[0]):
        if mat[i][j] > 0.5 and j not in removed:
            l1.append(news2[j])
            removed.append(j)
    buckets.append(l1)  

In [14]:
for i in buckets:
    if len(i) > 1:
        print i

["Live: SC verdict on triple talaq 'historic', says PM", 'Historic triple talaq verdict: Who said what', 'Full text: SC verdict on triple talaq']
["Taimur Ali Khan Stole The Thunder At Soha Ali Khan's Baby Shower ", 'Soha Ali Khan flaunts baby bump during yoga']
['2017 Hyundai Verna Launched In India. Price And Other Details Here', 'Hyundai launches 2017 Verna: All details here']
['AirAsia Offers Rs 999 Tickets In 7-Day Sale. Details Here ', 'AirAsia offers tickets starting from Rs 999']
["SC tells Tamil Nadu govt to follow NEET after Centre says it can't get exemption", "Follow NEET, Tamil Nadu Told After Centre Says It Can't Get Exemption"]


In [ ]:
#So basically you see the first news item? 
#That came in all 3 channels because it was important. (That list has 3 members)
#So you can say if something comes in more than one place it is important
#SO we are tryinng to split good news from crap news
# Who wants to know this shit? This Is Why You Crave Sugar Immediately After Your Meal</strong>
#KAYYYy? Yep :) 